# No.4 Toy model with static filters

Import modules.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle

Load image data.

In [2]:
with open('data/ORENIST.data', 'rb') as file:
    images, labels = pickle.load(file)

Define a function to create static filters.

In [3]:
def edge_filter():
    filter0 = np.array(
            [[ 2, 1, 0,-1,-2],
             [ 3, 2, 0,-2,-3],
             [ 4, 3, 0,-3,-4],
             [ 3, 2, 0,-2,-3],
             [ 2, 1, 0,-1,-2]]) / 23.0
    filter1 = np.array(
            [[ 2, 3, 4, 3, 2],
             [ 1, 2, 3, 2, 1],
             [ 0, 0, 0, 0, 0],
             [-1,-2,-3,-2,-1],
             [-2,-3,-4,-3,-2]]) / 23.0
    
    filter_array = np.zeros([5,5,1,2])
    filter_array[:,:,0,0] = filter0
    filter_array[:,:,0,1] = filter1

    return tf.constant(filter_array, dtype=tf.float32)

Define functions to apply filters.

In [4]:
x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv = edge_filter()
h_conv = tf.abs(tf.nn.conv2d(x_image, W_conv,
                             strides=[1,1,1,1], padding='SAME'))
h_conv_cutoff = tf.nn.relu(h_conv-0.2)

h_pool =tf.nn.max_pool(h_conv_cutoff, ksize=[1,2,2,1],
                       strides=[1,2,2,1], padding='SAME')

Define softmax classifer using the filtered images.

In [5]:
h_pool_flat = tf.reshape(h_pool, [-1, 392])

w0 = tf.Variable(tf.zeros([392, 3]))
b0 = tf.Variable(tf.zeros([3]))
p = tf.nn.softmax(tf.matmul(h_pool_flat, w0) + b0)

Define loss function, accuracy and optimizer.

In [6]:
t = tf.placeholder(tf.float32, [None, 3])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer().minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Create a session and initialize Variables.

In [7]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

Run the optimizer. It achieves the 100% accuracy with a single iteration.

In [8]:
sess.run(train_step, feed_dict={x:images, t:labels})
loss_val, acc_val = sess.run(
  [loss, accuracy], feed_dict={x:images, t:labels})
print ('Step: %d, Loss: %f, Accuracy: %f'
       % (1, loss_val, acc_val))

Step: 1, Loss: 96.313980, Accuracy: 1.000000
